In [ ]:
import subprocess

import gdal
import matplotlib.pyplot as plt
import numpy as np
from tdm.radar import utils

gdal.UseExceptions()

In [ ]:
class MemRasterBuilder(object):

    def __init__(self, geo_tr, wkt):
        self.driver = gdal.GetDriverByName("MEM")
        self.geo_tr = geo_tr
        self.wkt = wkt

    def build(self, data):
        rows, cols = data.shape
        raster = self.driver.Create("", cols, rows, 1, gdal.GDT_Float32)
        band = raster.GetRasterBand(1)
        band.WriteArray(data.filled())
        band.SetNoDataValue(float(data.fill_value))
        band.FlushCache()  # useless?
        raster.SetGeoTransform(self.geo_tr)
        raster.SetProjection(self.wkt)
        return raster

In [ ]:
def show_rain(rain):
    plt.cla()
    plt.clf()
    plt.figure(dpi=144)
    c = plt.contourf(rain, levels=np.arange(0, 3, 0.1))
    cbar = plt.colorbar(c)
    plt.gca().set_aspect("equal")

In [ ]:
def band_to_ma(band):
    assert band.GetMaskFlags() == gdal.GMF_NODATA
    m_band = band.GetMaskBand()
    return np.ma.masked_array(
        band.ReadAsArray(),
        mask=(m_band.ReadAsArray() == 0),
        fill_value=band.GetNoDataValue()
    )

In [ ]:
def read_rain(fn):
    dataset = gdal.Open(fn)
    return band_to_ma(dataset.GetRasterBand(1))

In [ ]:
def show_rain_from_file(fn):
    show_rain(read_rain(fn))

In [ ]:
def check_tif(fn, exp_rain, geo_tr, wkt):
    dataset = gdal.Open(fn)
    assert dataset.RasterCount == 1
    band = dataset.GetRasterBand(1)
    rain = band_to_ma(band)
    assert np.array_equal(rain.mask, exp_rain.mask)
    assert np.ma.allclose(rain, exp_rain)
    assert dataset.GetGeoTransform() == geo_tr
    assert dataset.GetProjectionRef() == wkt

### Compute rainfall and save to gtiff

In [ ]:
dt_str = "2018-05-01_23:00:04"
signal = utils.get_image_data(f"data/radarsample/cag01est2400/{dt_str}.png")
rain = utils.estimate_rainfall(signal)

In [ ]:
show_rain(rain)

In [ ]:
orig_fn = "rain.tif"
ga = utils.GeoAdapter("data/radarsample/radarfootprint.tif")
ga.save_as_gtiff(orig_fn, rain)
check_tif(orig_fn, rain, (ga.oX, ga.pxlW, 0, ga.oY, 0, ga.pxlH), ga.wkt)

In [ ]:
show_rain_from_file(orig_fn)

### Warp with `gdalwarp`

In [ ]:
t_srs_code = "EPSG:4326"
t_srs = gdal.osr.SpatialReference()
t_srs.ImportFromEPSG(int(t_srs_code.split(":")[1]))

In [ ]:
warped_fn = "warped_rain.tif"
subprocess.check_call(["gdalwarp", "-t_srs", t_srs_code, orig_fn, warped_fn])

In [ ]:
show_rain_from_file(warped_fn)

### Warp with the API

#### 1. Try `gdal.AutoCreateWarpedVRT`

In [ ]:
raster_builder = MemRasterBuilder((ga.oX, ga.pxlW, 0, ga.oY, 0, ga.pxlH), ga.wkt)
raster = raster_builder.build(rain)
resampling = gdal.GRA_NearestNeighbour
error_thr = 0.125
s_wkt = None  # use wkt from raster
warped_raster = gdal.AutoCreateWarpedVRT(raster, s_wkt, t_srs.ExportToWkt(), resampling, error_thr)
warped_rain = band_to_ma(warped_raster.GetRasterBand(1))
show_rain(warped_rain)

Looks the same, except for an ugly artifact on the top right

In [ ]:
dataset = gdal.Open(warped_fn)
print(dataset.RasterCount == 1)
print(dataset.GetGeoTransform() == warped_raster.GetGeoTransform())
print(dataset.GetProjectionRef() == warped_raster.GetProjectionRef())

WKTs are not exactly the same, but...

In [ ]:
dataset_srs = gdal.osr.SpatialReference(wkt=dataset.GetProjectionRef())
warped_raster_srs = gdal.osr.SpatialReference(wkt=warped_raster.GetProjectionRef())
assert warped_raster_srs.IsSame(dataset_srs)

In [ ]:
exp_warped_rain = band_to_ma(dataset.GetRasterBand(1))
print(warped_rain.shape == exp_warped_rain.shape)
print(warped_rain.fill_value == exp_warped_rain.fill_value)
print(np.ma.allclose(warped_rain, exp_warped_rain))
print(np.array_equal(warped_rain.mask, exp_warped_rain.mask))

In [ ]:
plt.cla()
plt.clf()
plt.figure(dpi=144)
plt.contourf(warped_rain.mask != exp_warped_rain.mask)
plt.gca().set_aspect("equal")

In [ ]:
(warped_rain != exp_warped_rain).sum() / warped_rain.size

In [ ]:
(abs(warped_rain.filled(0) - exp_warped_rain.filled(0)) > 1e-6).sum() / warped_rain.size

#### 2. Try `gdal.Warp`

In [ ]:
warped_raster_2 = gdal.Warp("", raster, format="MEM", dstSRS=t_srs_code)
warped_rain_2 = band_to_ma(warped_raster_2.GetRasterBand(1))
show_rain(warped_rain_2)

Again, it looks the same, but the top-right artifact is gone

In [ ]:
print(warped_raster_2.RasterCount == 1)
print(dataset.GetGeoTransform() == warped_raster_2.GetGeoTransform())
print(dataset.GetProjectionRef() == warped_raster_2.GetProjectionRef())

In [ ]:
warped_raster_2_srs = gdal.osr.SpatialReference(wkt=warped_raster_2.GetProjectionRef())
assert warped_raster_2_srs.IsSame(dataset_srs)

In [ ]:
print(warped_rain_2.shape == exp_warped_rain.shape)
print(warped_rain_2.fill_value == exp_warped_rain.fill_value)
print(np.ma.allclose(warped_rain_2, exp_warped_rain))
print(np.array_equal(warped_rain_2.mask, exp_warped_rain.mask))